<a href="https://colab.research.google.com/github/Rosefinch-Midsummer/Awesome-Colab/blob/master/DS/TGI%E5%88%86%E6%9E%90%E5%AE%9E%E6%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[TGI分析实战](https://github.com/seizeeveryday/DA-cases)

In [1]:
!wget https://github.com/seizeeveryday/DA-cases/raw/master/TGI/TGI%E6%8C%87%E6%95%B0%E6%A1%88%E4%BE%8B%E6%95%B0%E6%8D%AE.xlsx

--2020-01-02 07:54:01--  https://github.com/seizeeveryday/DA-cases/raw/master/TGI/TGI%E6%8C%87%E6%95%B0%E6%A1%88%E4%BE%8B%E6%95%B0%E6%8D%AE.xlsx
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/seizeeveryday/DA-cases/master/TGI/TGI%E6%8C%87%E6%95%B0%E6%A1%88%E4%BE%8B%E6%95%B0%E6%8D%AE.xlsx [following]
--2020-01-02 07:54:06--  https://raw.githubusercontent.com/seizeeveryday/DA-cases/master/TGI/TGI%E6%8C%87%E6%95%B0%E6%A1%88%E4%BE%8B%E6%95%B0%E6%8D%AE.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1959616 (1.9M) [application/octet-stream]
Saving to: ‘TGI指数案例数据.xlsx’

TGI指数案例数据.xls 100%[===========

In [0]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('TGI指数案例数据.xlsx')
df.head()

,品牌名称,买家昵称,付款日期,订单状态,实付金额,邮费,省份,城市,购买数量
0,viva la vida,做快淘饭,2019-04-18 00:03:00,交易成功,22.32,0,北京,北京市,1
1,viva la vida,作自有世祟,2019-02-17 00:03:51,交易成功,87.00,0,上海,上海市,1
2,viva la vida,作雪白室,2019-04-18 00:01:43,交易成功,97.66,0,福建省,福州市,2
3,viva la vida,作美女购物主,2019-01-11 23:35:01,交易成功,37.23,0,河南省,安阳市,3
4,viva la vida,作美女购物主,2019-02-18 14:16:03,交易成功,29.50,0,河南省,安阳市,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28832 entries, 0 to 28831
Data columns (total 9 columns):
品牌名称    28832 non-null object
买家昵称    28832 non-null object
付款日期    28832 non-null datetime64[ns]
订单状态    28832 non-null object
实付金额    28832 non-null float64
邮费      28832 non-null int64
省份      28832 non-null object
城市      28832 non-null object
购买数量    28832 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 2.0+ MB


In [5]:
#计算单个用户平均支付金额
gp_user = df.groupby('买家昵称')['实付金额'].mean().reset_index()
gp_user.head()

,买家昵称,实付金额
0,.blue_ram,49.450
1,.christiny,22.000
2,.willn1,34.570
3,.托托m,37.475
4,0000妮,13.500


In [6]:
#基于用户支付金额，判断用户是属于低客单还是高客单
def if_high(x):
    if x > 50:
        return '高客单'
    else:
        return '低客单'

gp_user['客单类别'] = gp_user['实付金额'].apply(if_high)
gp_user.head(10)

,买家昵称,实付金额,客单类别
0,.blue_ram,49.450,低客单
1,.christiny,22.000,低客单
2,.willn1,34.570,低客单
3,.托托m,37.475,低客单
4,0000妮,13.500,低客单
5,0009797王,94.500,高客单
6,000xyx0,99.250,高客单
7,000米粒儿米粒0,24.500,低客单
8,00556旭79618,23.860,低客单
9,00不哭0,53.545,高客单


In [7]:
#将客单数据和地域数据合并
#先去重
df_dup = df.loc[df.duplicated('买家昵称') == False,:]

#再合并
df_merge = pd.merge(gp_user,df_dup,left_on = '买家昵称',right_on = '买家昵称',how = 'left')
df_merge.head()

,买家昵称,实付金额_x,客单类别,品牌名称,付款日期,订单状态,实付金额_y,邮费,省份,城市,购买数量
0,.blue_ram,49.450,低客单,viva la vida,2019-02-04 17:49:34.000,交易成功,49.450,0,上海,上海市,1
1,.christiny,22.000,低客单,viva la vida,2019-01-29 14:17:15.000,交易成功,22.000,0,江苏省,南京市,1
2,.willn1,34.570,低客单,viva la vida,2019-01-11 03:46:18.000,交易成功,34.570,0,山东省,烟台市,2
3,.托托m,37.475,低客单,viva la vida,2019-01-11 02:26:33.000,交易成功,37.475,0,上海,上海市,3
4,0000妮,13.500,低客单,viva la vida,2019-06-28 16:53:26.458,交易成功,13.500,0,广东省,揭阳市,1


In [8]:
#用透视表的方法来统计每个省市低客单、高客单人数
#先筛选出我们需要的列
df_merge = df_merge[['买家昵称','客单类别','省份','城市']]

#再用透视表
result = pd.pivot_table(df_merge,index =['省份','城市'],columns = '客单类别',aggfunc = 'count')
result.head()

买家昵称        
客单类别            低客单     高客单
省份  城市                     
上海  上海市      2818.0  2374.0
云南省 临沧市         3.0     2.0
    丽江市         1.0     3.0
    保山市         6.0     2.0
    大理白族自治州     9.0     8.0

In [9]:
#将低客单和高客单数据转化为我们熟悉的DF格式
tgi = pd.merge(result['买家昵称']['高客单'].reset_index(),result['买家昵称']['低客单'].reset_index(),
               left_on = ['省份','城市'],right_on = ['省份','城市'],how = 'inner')
tgi.head()

,省份,城市,高客单,低客单
0,上海,上海市,2374.0,2818.0
1,云南省,临沧市,2.0,3.0
2,云南省,丽江市,3.0,1.0
3,云南省,保山市,2.0,6.0
4,云南省,大理白族自治州,8.0,9.0


In [10]:
#计算总人数，以及每个城市对应的高客单占比
tgi['总人数'] = tgi['高客单'] + tgi['低客单']
tgi['高客单占比'] = tgi['高客单'] / tgi['总人数']

tgi.head()

,省份,城市,高客单,低客单,总人数,高客单占比
0,上海,上海市,2374.0,2818.0,5192.0,0.457242
1,云南省,临沧市,2.0,3.0,5.0,0.400000
2,云南省,丽江市,3.0,1.0,4.0,0.750000
3,云南省,保山市,2.0,6.0,8.0,0.250000
4,云南省,大理白族自治州,8.0,9.0,17.0,0.470588


In [11]:
#检核数据空值情况
tgi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 346 entries, 0 to 345
Data columns (total 6 columns):
省份       346 non-null object
城市       346 non-null object
高客单      332 non-null float64
低客单      329 non-null float64
总人数      315 non-null float64
高客单占比    315 non-null float64
dtypes: float64(4), object(2)
memory usage: 18.9+ KB


In [0]:
#去除空值
tgi = tgi.dropna()

In [13]:
#计算总体高客单人数占比
total_percentage = tgi['高客单'].sum() / tgi['总人数'].sum()
total_percentage

0.41528303343887557

In [14]:
#计算每个城市高客单TGI指数
tgi['高客单TGI指数'] = tgi['高客单占比'] / total_percentage * 100
tgi = tgi.sort_values('高客单TGI指数',ascending = False)
tgi.head(10)

,省份,城市,高客单,低客单,总人数,高客单占比,高客单TGI指数
149,新疆维吾尔自治区,哈密市,4.0,1.0,5.0,0.800000,192.639702
152,新疆维吾尔自治区,巴音郭楞蒙古自治州,10.0,3.0,13.0,0.769231,185.230483
2,云南省,丽江市,3.0,1.0,4.0,0.750000,180.599721
277,甘肃省,白银市,3.0,1.0,4.0,0.750000,180.599721
34,吉林省,辽源市,2.0,1.0,3.0,0.666667,160.533085
44,四川省,广安市,6.0,3.0,9.0,0.666667,160.533085
136,广西壮族自治区,河池市,4.0,2.0,6.0,0.666667,160.533085
25,内蒙古自治区,锡林郭勒盟,2.0,1.0,3.0,0.666667,160.533085
343,黑龙江省,鹤岗市,2.0,1.0,3.0,0.666667,160.533085
97,山西省,临汾市,9.0,5.0,14.0,0.642857,154.799761


In [15]:
#筛选出人数大于平均值的人数，再计算更合理的TGI指数
tgi.loc[tgi['总人数'] > tgi['总人数'].mean(),:].head(10)

,省份,城市,高客单,低客单,总人数,高客单占比,高客单TGI指数
287,福建省,福州市,145.0,135.0,280.0,0.517857,124.699807
124,广东省,珠海市,49.0,52.0,101.0,0.485149,116.823582
27,北京,北京市,1203.0,1298.0,2501.0,0.481008,115.826450
283,福建省,厦门市,105.0,118.0,223.0,0.470852,113.380991
111,广东省,佛山市,118.0,135.0,253.0,0.466403,112.309708
173,江西省,南昌市,63.0,73.0,136.0,0.463235,111.546887
46,四川省,成都市,287.0,334.0,621.0,0.462158,111.287429
0,上海,上海市,2374.0,2818.0,5192.0,0.457242,110.103682
164,江苏省,无锡市,135.0,162.0,297.0,0.454545,109.454376
120,广东省,深圳市,438.0,528.0,966.0,0.453416,109.182440
